# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [2]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [3]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [4]:
aqdet.LoadDetectors(['taggerNewer.json'])

taggerNewer.json


We set up histograms for each of the channels, and then fill them with tagger times from our data:

In [5]:
taggerChannels = aqdet.Channels['Tagger']                 # number of channels in the tagger (368)

histos = [None]*taggerChannels          # set up an array of 368 histograms to fill      

for i in range(taggerChannels):         # for each tagger channel
    histos[i] = Hist(1300,-500,800)     # set up a histogram for that tagger channel

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)
    taggerTimes = aqdet.Arrays['Tagger']['Time']
    
    for dat in taggerTimes:
        for time in dat['Time']:
            for i in range(taggerChannels):
                if (dat['channel']==i):
                    histos[i].Fill(time)
                    
    if(aq.eventNo%5000==0):
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(plotCalTagger,0,500000)

('Events Processed: ', 5000)
('Events Processed: ', 10000)
('Events Processed: ', 15000)
('Events Processed: ', 20000)
('Events Processed: ', 25000)
('Events Processed: ', 30000)
('Events Processed: ', 35000)
('Events Processed: ', 40000)
('Events Processed: ', 45000)
('Events Processed: ', 50000)
('Events Processed: ', 55000)
('Events Processed: ', 60000)
('Events Processed: ', 65000)
('Events Processed: ', 70000)
('Events Processed: ', 75000)
('Events Processed: ', 80000)
('Events Processed: ', 85000)
('Events Processed: ', 90000)
('Events Processed: ', 95000)
('Events Processed: ', 100000)
('Events Processed: ', 105000)
('Events Processed: ', 110000)
('Events Processed: ', 115000)
('Events Processed: ', 120000)
('Events Processed: ', 125000)
('Events Processed: ', 130000)
('Events Processed: ', 135000)
('Events Processed: ', 140000)
('Events Processed: ', 145000)
('Events Processed: ', 150000)
('Events Processed: ', 155000)
('Events Processed: ', 160000)
('Events Processed: ', 16500

Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them:

In [9]:
ROOT.enableJSVis()

tagger_times = ROOT.TFile("tagger_times.root","RECREATE")


myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")                   # make a gaussian fit with an offset/background
myFit.SetParNames("constant","mean","stdev","background")     # label the parameters


peak = [None]*taggerChannels
sigma = [None]*taggerChannels
bgLevel = [None]*taggerChannels

pk = 140
sig = 1.1
bg = 100


for i in range(taggerChannels):                                        
    myFit.SetParameter(1,pk)                 
    myFit.SetParameter(2,sig)                
    myFit.SetParameter(3,bg)                
    
    histos[i].SetTitle("Channel "+str(i))      
    histos[i].Draw()                
    
    if(histos[i].GetEntries()>10):                                     
        histos[i].Fit('myFit','SQ','',-300,600)                           
        peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)
        sigma[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2)
        bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3) 
        
        k=1
        while(ROOT.gMinuit.fCstatu!="CONVERGED "):                                     
            myFit.SetParameter(1,145-k)                 
            myFit.SetParameter(2,1)                   
            myFit.SetParameter(3,80-k)
            
            peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)
            sigma[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2)
            bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3) 
            
            k+=1
            
        pk = peak[i]
        sig = sigma[i]
        bg = bgLevel[i]
        
        if (peak[i]>180 or peak[i]<100):
            print("Check channel "+ str(i)+" for peak")
            print(i,k,pk,sig,bg)
        if (sigma[i]<-5):
            print("Check channel "+ str(i)+" for sigma")
            print(i,k,pk,sig,bg)
            
    else:                                            
        print("Histogram "+str(i)+" is empty (has "+str(histos[i].GetEntries())+" entries).")      
     
    histos[i].Write("tagger_times")    
    
    

Check channel 39 for sigma
(39, 1, 111.7557543200525, -7.2037017650188115, 20.81258366963532)
Check channel 40 for peak
(40, 1, 290.5889535667818, -63.126704473321276, 18.93971424949334)
Check channel 40 for sigma
(40, 1, 290.5889535667818, -63.126704473321276, 18.93971424949334)
Check channel 42 for peak
(42, 1, 187.25577884628433, 0.5048980598116183, 18.316545535095237)
Check channel 195 for peak
(195, 3, 43.45611267260515, -16.800632880160194, 2.445988145897738)
Check channel 195 for sigma
(195, 3, 43.45611267260515, -16.800632880160194, 2.445988145897738)
Check channel 196 for peak
(196, 1, 42.84138006877663, -2.2845488908148637, 2.3667289479597495)
Check channel 197 for peak
(197, 1, 43.379957025620314, -1.041969477823702, 2.400424800909461)
Check channel 198 for peak
(198, 1, 44.97940286629268, 0.824637188387248, 2.2625608532562973)
Check channel 199 for sigma
(199, 4, 143.08063749978524, -15.46117722304201, 2.2158395998530267)
Check channel 200 for sigma
(200, 1, 111.72420317695

Finding how much each peak is off from zero, and setting it back to zero (not worked on yet):

In [ ]:
bins = 1200
ROOT.enableJSVis()
c2 = ROOT.TCanvas("my2","Title",1000,2500)
c2.Divide(2,5)

histos2 = [None]*10
diff = [None]*10
for i in range(10):
    histos2[i] = Hist(bins,300,1500)
    diff[i] = int(round(means[i]-800))

for i in range(10):
    for j in range(bins):
        histos2[i].SetBinContent(j-diff[i],histos2[i].GetBinContent(j))
                 
    c2.cd(i+1)
    histos2[i].SetTitle("Channel "+str(i))
    histos2[i].Draw("colz")
    myFit.SetParameter(1,800)
    myFit.SetParameter(2,5)
    histos2[i].Fit('myFit')
    
c2.Draw()